In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mathinstruct-dataset-hybrid-math-instruction-tun/train.csv


In [2]:
%pip install -U datasets # To convert dataset into huggingface dataset format
%pip install accelerate # 
%pip install peft # For finetuning and LoRa 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 8.0 MB/s eta 0:00:00ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


# Load model and Tokenizer

In [3]:
import torch
import transformers
from transformers import (AutoModelForCausalLM,AutoTokenizer, BitsAndBytesConfig,
                          HfArgumentParser,TrainingArguments,Pipeline,logging)

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

In [4]:
model_id = "databricks/dolly-v2-3b"

In [5]:
# Tokenizer

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

"""
By setting tokenizer.pad_toke = tokenizer.eos_token,
The code is instructing the tokenizer to ues the end-of-sequence token as the padding token.
"""

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/228 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


'\nBy setting tokenizer.pad_toke = tokenizer.eos_token,\nThe code is instructing the tokenizer to ues the end-of-sequence token as the padding token.\n'

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_id, 
#                                              use_cache = False,
                                            device_map = "auto", # balance
                                            load_in_8bit = False, # NO Quantized model we need here 
                                            torch_dtype = torch.float16)

config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.68G [00:00<?, ?B/s]

# Import Required Libraries

In [7]:
import pandas as pd
import os
import random
from datasets import Dataset,disable_caching
from typing import Dict,List
from copy import deepcopy
from functools import partial
from transformers import DataCollatorForSeq2Seq

disable_caching()

# Dataset Preparation

In [8]:
df = pd.read_csv('/kaggle/input/mathinstruct-dataset-hybrid-math-instruction-tun/train.csv')
df.head()

,source,instruction,output
0,data/CoT/aqua_rat.json,The distance between two stars is 6.52 × 10^5 ...,Let's think about the multi-choice question.\n...
1,data/CoT/aqua_rat.json,How many ways can the letters in the word COMM...,Let's solve the multi-choice question step by ...
2,data/PoT/aqua_rat_filtered.json,A team of six entered for a shooting competiti...,"answers = ['A', 'B', 'C', 'D', 'E']\n# If the ..."
3,data/CoT/gsm_rft.json,A psychiatrist has 4 patients that need 25 ses...,The second patient needs 6+5 = 11 sessions\n25...
4,data/PoT/aqua_rat_filtered.json,The radius of a wheel is 22.4 cm. What is the ...,radius = 22.4\nresolutions = 500\n# calculate ...


In [9]:
prompt = """Below is an instruction and output. Understand  the Instruction and accordingly give the ouput. \n\nInstruction : {instruction}\n\n"""

answer = """Answer : {output}"""

In [10]:
def data_preprocess(df):
    def error_raise(col_name):
        raise ValueError(f"Expected an {col_name} in dataframe")
    instruction = df['instruction']
    output = df["output"]
    if not instruction:
        error_raise(instruction)
    if not output:
        error_raise(output)
    prompt1 = prompt.format(instruction = instruction)
    answer1 = answer.format(output = output)
    df["text"] = prompt1 + answer1
    return df

In [11]:
df.columns


Index(['source', 'instruction', 'output'], dtype='object')

In [12]:
## Taking small dataset from whole dataset which is 1% of whole data 
index = random.sample(range(df.shape[0]),round(df.shape[0]*0.01))
small_dataset = df.iloc[index]
small_dataset.reset_index(inplace=True)

In [13]:
small_dataset = small_dataset.apply(data_preprocess,axis=1)

In [14]:
print(small_dataset.text[0])

Below is an instruction and output. Understand  the Instruction and accordingly give the ouput. 

Instruction :  "statement 1": 180 , calculate the dividend from Carol ' s stock ., "statement 2" :less than 880 , calculate the dividend from Carol ' s stock ., "options: " Entailment or contradiction or neutral? Let's program in Python in the response.

Answer : carol_stock_s1 = 180
carol_stock_s2 = 880
 
if carol_stock_s1 is None or carol_stock_s2 is None:
 print('neutral')

elif carol_stock_s1 < carol_stock_s2:
 print('Entailment')

elif carol_stock_s1 != carol_stock_s2:
 print('contradiction')


In [15]:
hf_dataset = Dataset.from_pandas(small_dataset)
hf_dataset

Dataset({
    features: ['index', 'source', 'instruction', 'output', 'text'],
    num_rows: 2623
})

In [16]:
MAX_LENGTH = 512

In [17]:
def _preprocessing_batch(batch:Dict[str,List]):
    model_input  = tokenizer(batch["text"],max_length=MAX_LENGTH, 
                            truncation=True, padding="max_length"
                            )
    model_input["labels"] = deepcopy(model_input["input_ids"])
    return model_input

In [18]:
preprocessing_fuc = partial(_preprocessing_batch)

In [19]:
df.columns

Index(['source', 'instruction', 'output'], dtype='object')

In [20]:
encoded_small_dataset = hf_dataset.map(
preprocessing_fuc, 
batched = True,
remove_columns = ["instruction","source","output"])

Map:   0%|          | 0/2623 [00:00<?, ? examples/s]

In [21]:
processed_dataset = encoded_small_dataset.filter(lambda rec : len(rec["input_ids"])<= MAX_LENGTH)


Filter:   0%|          | 0/2623 [00:00<?, ? examples/s]

In [22]:
# among data taken 10 % for testing 
split_dataset = processed_dataset.train_test_split(test_size = round(small_dataset.shape[0]*.1), seed = 0)


In [23]:
print(split_dataset)


DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2361
    })
    test: Dataset({
        features: ['index', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 262
    })
})


In [24]:
data_collator = DataCollatorForSeq2Seq(model = model,
                                      tokenizer= tokenizer, 
                                      max_length=MAX_LENGTH,
                                      pad_to_multiple_of=8,
                                      padding="max_length")

# Load Model in LoRA (Low Rank Adaptation)

In [25]:
from peft import LoraConfig,get_peft_model, prepare_model_for_kbit_training


In [26]:
## LoRA Setting parameters
LORA_R = 512 #512
LORA_ALPHA = 512 #1024
LORA_DROPOUT = 0.05 

In [27]:
# LORA CONFIGRIGATION
lora_config = LoraConfig(r = LORA_R, # the diamension of the low-rank metrics
                        lora_alpha=LORA_ALPHA, # scaling factor for the weight matrices
                        lora_dropout= LORA_DROPOUT, # dropout probability of the lora layers
                        bias= "none",
                        task_type="CAUSAL_LM",
                        target_modules=["query_key_value"],
                        )

In [28]:
# prepare int-8 model for training- utility function that prepare a pytorch model for int8 question
model = prepare_model_for_kbit_training(model,)

In [29]:
# initialize the model with the lora framework
model = get_peft_model(model, lora_config)

In [30]:
model.print_trainable_parameters()


trainable params: 167,772,160 || all params: 2,942,858,240 || trainable%: 5.7010


# Finetuning

In [31]:
from transformers import TrainingArguments, Trainer

In [32]:
EPOCHS = 3
LR = 1e-4
MODEL_SAVE_FOLDER_PATH = "dolly-3b-lora"

In [33]:
training_arg = TrainingArguments(
                output_dir=MODEL_SAVE_FOLDER_PATH,
                overwrite_output_dir=True,
                fp16=True, # convert to float precision 16 using bitsandbytes
                per_device_train_batch_size=1,
                per_device_eval_batch_size=1,
                learning_rate=LR,
                num_train_epochs=EPOCHS,
                logging_strategy= "epoch",
                eval_strategy="epoch",
                save_strategy="epoch",
                )

In [34]:
trainer = Trainer(
model=model,
tokenizer=tokenizer,
    args= training_arg,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    data_collator=data_collator,
    )

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [35]:
model.config.use_cache = False # silence the warning, please re-enable for inference!


In [36]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.510100,0.436621
2,0.370600,0.433637
3,0.248800,0.472399


TrainOutput(global_step=7083, training_loss=0.37651456461002575, metrics={'train_runtime': 5589.1354, 'train_samples_per_second': 1.267, 'train_steps_per_second': 1.267, 'total_flos': 6.123283605356544e+16, 'train_loss': 0.37651456461002575, 'epoch': 3.0})

In [37]:
# Save finetuned model
trainer.save_model(MODEL_SAVE_FOLDER_PATH)
trainer.model.config.save_pretrained(MODEL_SAVE_FOLDER_PATH)

# Prediction on Real Data

In [38]:
from transformers import pipeline


In [39]:
def postprocess(response):
    messages = response.split("Response:")
    if not messages:
        raise ValueError("Invalid template for prompt. The template should include the term **Response**")
    return "".join(messages[1:])

In [44]:
df.instruction.iloc[-10]

'Here is a numerical problem for a math student in Algebraic Geometry- Cohomology of Algebraic Varieties:\n\nLet X be the algebraic variety defined by the equation x^2 + y^2 - 1 = 0 in the complex plane. Calculate the dimension of the cohomology group H^1(X, ℂ) where ℂ is the field of complex numbers.'

In [45]:
instruction = 'Here is a numerical problem for a math student in Algebraic Geometry- Cohomology of Algebraic Varieties:\n\nLet X be the algebraic variety defined by the equation x^2 + y^2 - 1 = 0 in the complex plane. Calculate the dimension of the cohomology group H^1(X, ℂ) where ℂ is the field of complex numbers.'


inf_pipeline = pipeline(task = "text-generation",model = trainer.model,
                        tokenizer = tokenizer, max_length = MAX_LENGTH)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

In [48]:
response = inf_pipeline(prompt.format(instruction = instruction))[0]['generated_text']

In [52]:
print(response.split("Answer :")[-1])

 To calculate the dimension of the cohomology group H^1(X, ℂ), we will use the Leray spectral sequence. Let's first recall the Leray spectral sequence.

The Leray spectral sequence is a sequence of cohomology groups that arises from the fibration of the variety X over the base field K. The first term in the sequence is H^0(X_b, ℂ), where X_b is the base field and ℂ is the field of complex numbers. The second term is H^1(X_b, ℂ), and the third term is H^2(X_b, ℂ).

Now, let's consider the fibration X → B → K, where B is the base field and K is the field of complex numbers. The Leray spectral sequence will have the following terms:

1. H^0(X_b, ℂ) → H^1(X_b, ℂ) → H^2(X_b, ℂ)
2. H^1(X_b, ℂ) → H^2(X_b, ℂ) → H^3(B, ℂ)
3. H^2(X_b, ℂ) → H^3(B, ℂ) → H^4(B, ℂ)

Since we are working over the field of complex numbers, we can use the fact that the Leray spectral sequence is a fibration with base field K. Therefore, the third term is 0, and the second term is the cohomology group we are interested 